# Sistema de recomendacion

## Importación de librerias

In [2]:
 # Importamos las librerias correspondientes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Importación de datasets

In [3]:
# Leemos los dataset exportados con anterioridad
games= pd.read_parquet('Dataset_ETL/steam_games.parquet',engine='pyarrow')
reviews=pd.read_parquet('Dataset_ETL/user_reviews.parquet',engine='pyarrow')
items=pd.read_parquet('Dataset_ETL/users_items.parquet',engine='pyarrow')

In [6]:
games.head()

,publisher,genres,app_name,release_date,price,id,developer,average price
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,3.473871
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,25.29,643980,Secret Level SRL,25.290000
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域,6.607647
4,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,3.99,772540,Trickjump Games Ltd,7.823333
5,Poppermost Productions,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Basic Pass,2018-01-04,9.99,774276,Poppermost Productions,26.790000


In [5]:
reviews.head()

,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,2
2,76561197970982479,43110,True,1
3,js41637,251610,True,2
4,js41637,227300,True,1


In [9]:
recommends = pd.merge(reviews,games,left_on='item_id',right_on='id')

In [11]:
recommends=recommends.iloc[:,[0,1,3,6]]

In [54]:
recommends = recommends[recommends.sentiment_analysis == 1]

In [55]:
recommends

,user_id,item_id,sentiment_analysis,app_name
0,76561197970982479,1250,1,Killing Floor
1,76561197970982479,22200,1,Zeno Clash
4,evcentric,248820,1,Risk of Rain
5,evcentric,370360,1,TIS-100
6,evcentric,237930,1,Transistor
...,...,...,...,...
49074,76561198312638244,130,1,Half-Life: Blue Shift
49075,76561198312638244,70,1,Half-Life
49076,76561198312638244,362890,1,Black Mesa
49078,LydiaMorley,730,1,Counter-Strike: Global Offensive


In [53]:
recommends.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49080 entries, 0 to 49079
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             49080 non-null  object
 1   item_id             49080 non-null  object
 2   sentiment_analysis  49080 non-null  int64 
 3   app_name            49080 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


In [37]:
# Convertir la escala 2, 1, 0 a 1, 0, -1
recommends.sentiment_analysis = recommends.sentiment_analysis.map({2:1,1:0,0:-1})

In [56]:
# Concatenar los valores duplicados en una cadena
df_pivot = recommends.pivot_table(index='user_id', columns='app_name', values='sentiment_analysis', aggfunc=lambda x: ', '.join(map(str, x)))

In [57]:
df_pivot.fillna(0,inplace=True)

In [58]:
df_pivot

app_name,//N.P.P.D. RUSH//- The milk of Ultraviolet,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,140,18 Wheels of Steel: Extreme Trucker,1943 Megami Strike,...,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theHunter Classic,theHunter: Primal,Астролорды: Облако Оорта
user_id,,,,,,,,,,,,,,,,,,,,,
--ace--,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-2SV-vuLB-Kg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-I_AM_EPIC-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-Kenny,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-Mad-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zv_odd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zvanik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zwanzigdrei,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
print(df_pivot.iloc[1,:].value_counts())

-2SV-vuLB-Kg
0    2017
1       3
Name: count, dtype: int64


In [52]:
print(df_pivot.iloc[0,:].dtype)

object


In [ ]:
df_pivot.iloc[:,0].astype('int')

In [46]:
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23005 entries, --000-- to zyr0n1c
Columns: 2923 entries, ! That Bastard Is Trying To Steal Our Gold ! to Астролорды: Облако Оорта
dtypes: object(2923)
memory usage: 513.2+ MB


He podido realizar la tabla con los datos de lso usuarios y con la parte de las calificaciones
Toca realizar en el dataset recommends, la similitud del coseno, esto hara que los usuarios tengan recomendaciones de juegos similares con una lista de 5 juegos similares que puedan gustarle.

## Similitud del coseno